In [2]:
import pandas as pd

In [3]:
# Load your final dataset (replace with your actual file path)
file1_path = "IHME-GBD_2021_DATA-5503ed84-1.csv"  # Risk factors
file2_path = "IHME-GBD_2021_DATA-6f8f7ceb-1.csv"  # Cause of death

file_air_pollution = "air-pollution.csv"
file_co2 = "co2 Emission Africa.csv"
file_haq = "IHME_GBD_2019_HAQ_1990_2019_DATA.CSV"

# Load datasets
df_risk = pd.read_csv(file1_path)
df_cause = pd.read_csv(file2_path)
df_air = pd.read_csv(file_air_pollution)
df_co2 = pd.read_csv(file_co2)
df_haq = pd.read_csv(file_haq)

# Load the datasets
air_pollution_df = pd.read_csv("air-pollution.csv")
co2_emission_df = pd.read_csv("co2 Emission Africa.csv")

In [8]:
# Dataset 1: Health Burden (Cause of Death Only)
# drop the columns not needed.
df_health_burden = df_cause.drop(columns=['sex','age','upper', 'lower'])

df_health_burden.head()


,measure,location,cause,metric,year,val
0,Deaths,Eritrea,Asthma,Number,2000,289.486429
1,Deaths,Eritrea,Interstitial lung disease and pulmonary sarcoi...,Number,2000,13.127577
2,Deaths,Eritrea,Other chronic respiratory diseases,Number,2000,37.860056
3,Deaths,Democratic Republic of the Congo,Chronic obstructive pulmonary disease,Number,2001,7642.356241
4,Deaths,Senegal,Chronic obstructive pulmonary disease,Number,2003,860.605027


In [11]:
# Dataset 2: Risk Factor Burden
df_risk_factor = df_risk.drop(columns=['sex','age','upper', 'lower'])
df_risk_factor = df_risk_factor.rename(columns={'rei': 'risk_factor'})


df_risk_factor.head()

,measure,location,cause,risk_factor,metric,year,val
0,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2000,257.840865
1,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2001,249.595886
2,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2002,245.426604
3,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2003,268.556783
4,Deaths,Morocco,Chronic obstructive pulmonary disease,Ambient ozone pollution,Number,2004,298.192621


In [14]:
# Dataset 3: Contextual Dataset
african_countries = [
    'Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi',
    'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros',
    'Congo', 'Côte d’Ivoire', 'Democratic Republic of the Congo', 'Djibouti',
    'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon',
    'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia',
    'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco',
    'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe',
    'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa',
    'South Sudan', 'Sudan', 'Togo', 'Tunisia', 'Uganda', 'United Republic of Tanzania',
    'Zambia', 'Zimbabwe'
]


54

In [23]:
#filter the health access and quality index for african countries

# Filter the df_haq DataFrame for the required indicators and countries
# Apply filtering criteria
df_haq_filtered = df_haq[
    (df_haq['location_name'].isin(african_countries)) &
    (df_haq['age_group_name'] == 'Age-standardized') &
    (df_haq['indicator_name'].isin(['Chronic respiratory diseases', 'HAQ Index'])) &
    (df_haq['year_id'] >= 2000)
][['location_name', 'year_id', 'indicator_name', 'val']]

# Pivot the table to create separate columns for each indicator
df_haq_pivot = df_haq_filtered.pivot_table(
    index=['location_name', 'year_id'],
    columns='indicator_name',
    values='val',
    aggfunc='mean'  # Using mean in case of duplicates
).reset_index()

# Rename columns
df_haq_pivot = df_haq_pivot.rename(columns={
    'location_name': 'Country',
    'year_id': 'Year',
    'Chronic respiratory diseases': 'Chronic_Respiratory(RSD)',
    'HAQ Index': 'HAQ_Index'
})

# Display the final dataset
print(df_haq_pivot.head())

indicator_name       Country  Year  Chronic_Respiratory(RSD)  HAQ_Index
0                    Algeria  2019                  0.000009  58.663453
1                     Angola  2019                  0.000027  29.302573
2                      Benin  2019                  0.000040  31.403459
3                   Botswana  2019                  0.000035  37.506129
4               Burkina Faso  2019                  0.000058  28.462344


In [26]:
# Merge on 'Country' and 'Year'
df_air_pol = pd.merge(df_co2, df_air, on=["Country", "Year"], how="inner")

# Calculate Population Density
df_air_pol["Population Density"] = df_air_pol["Population"] / df_air_pol["Area (Km2)"]

# Select and reorder columns to match the final structure
final_columns_order = [
    "Country", "Sub-Region", "Year", "Population", "GDP PER CAPITA (USD)", "Area (Km2)",
    "Population Density", "Total CO2 Emission excluding LUCF (Mt)", "Nitrogen Oxide",
    "Sulphur Dioxide", "Carbon Monoxide", "Organic Carbon", "NMVOCs", "Black Carbon", "Ammonia"
]

# Create the final DataFrame
df_air_pol = df_air_pol[final_columns_order]


df_air_pol.head()


,Country,Sub-Region,Year,Population,GDP PER CAPITA (USD),Area (Km2),Population Density,Total CO2 Emission excluding LUCF (Mt),Nitrogen Oxide,Sulphur Dioxide,Carbon Monoxide,Organic Carbon,NMVOCs,Black Carbon,Ammonia
0,Algeria,Northern Africa,2000,30774621,1780.38,2381741,12.921061,80.05,242832.24,89269.27,828892.97,94865.79,705727.43,48943.58,82763.81
1,Algeria,Northern Africa,2001,31200985,1754.58,2381741,13.100075,78.65,246238.80,83231.85,790778.92,94582.10,686010.13,49104.58,83028.51
2,Algeria,Northern Africa,2002,31624696,1794.81,2381741,13.277974,82.40,271977.39,108062.85,804976.24,95204.37,704868.58,52206.27,83399.16
3,Algeria,Northern Africa,2003,32055883,2117.05,2381741,13.459013,88.19,276968.15,99115.59,795145.20,95682.78,743752.69,55580.08,85153.41
4,Algeria,Northern Africa,2004,32510186,2624.80,2381741,13.649757,89.49,290319.45,105658.06,785143.58,96297.88,741058.65,58368.20,86709.51


In [ ]:
# First, filter the first dataset (co2_em) for African countries and the given year range 2000-2021.
african_countries = co2_em['Sub-Region'].notnull()  # African countries have valid Sub-Regions
year_range = (co2_em['Year'] >= 2000) & (co2_em['Year'] <= 2021)

co2_em_filtered = co2_em[african_countries & year_range]

# Now, filter the second dataset (air_pol) for the same year range 2000-2021.
year_range_2 = (air_pol['Year'] >= 2000) & (air_pol['Year'] <= 2021)
air_pol_filtered = air_pol[year_range_2]

# Merge the two datasets on 'Country' and 'Year'.
co2_air_merged = pd.merge(co2_em_filtered, air_pol_filtered, on=['Country', 'Year'], how='inner')




In [ ]:
# Grouping the dataset by Sub-Region and Year, then calculating the sum for each relevant feature
grouped_data = co2_air_merged.groupby(['Sub-Region', 'Year']).agg({
    'Population': 'sum',
    'GDP PER CAPITA (USD)': 'mean',  # GDP per capita should be averaged, not summed
    'Area (Km2)': 'mean',  # Area will not change within Sub-Region, so take the mean
    'Total CO2 Emission excluding LUCF (Mt)': 'sum',
    'Nitrogen Oxide': 'sum',
    'Sulphur Dioxide': 'sum',
    'Carbon Monoxide': 'sum',
    'Organic Carbon': 'sum',
    'NMVOCs': 'sum',
    'Black Carbon': 'sum',
    'Ammonia': 'sum'
}).reset_index()

# Calculating Population Density as Population divided by Area
grouped_data['Population Density'] = grouped_data['Population'] / grouped_data['Area (Km2)']

# Selecting the required features for the final dataset
final_data = grouped_data[[
    'Sub-Region', 'Year', 'Population', 'GDP PER CAPITA (USD)', 'Area (Km2)', 'Population Density',
    'Total CO2 Emission excluding LUCF (Mt)', 'Nitrogen Oxide', 'Sulphur Dioxide', 'Carbon Monoxide',
    'Organic Carbon', 'NMVOCs', 'Black Carbon', 'Ammonia'
]]

# Display the final dataset
tools.display_dataframe_to_user(name="Final Dataset with Population Density and Emissions by Sub-Region", dataframe=final_data)
